In [1]:
import geopandas as gpd
import shapely
from getpass import getpass

import sys
import copy
sys.path.append("../Code")

from helper_functions import *
from loading_data import *
# from algorithms import *

WelkomCorne!


In [ ]:
dfob= get_dataframe("""
                    SELECT bk_votpand_cluster, COUNT(*)
                    FROM proj_afval_netwerk.rel_votpand_cluster_verblijfsobject
                    GROUP BY bk_votpand_cluster
                    """)

In [ ]:
dfob['split'] = dfob['bk_votpand_cluster'].str.split('~')
dfob['bag'] = dfob['split'].apply(lambda x: x[0]).astype('int64')
dfob['x'] = dfob['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
dfob['y'] = dfob['split'].apply(lambda x: x[2]).astype('float').round().astype('int')
dfob = dfob.drop(['split'], axis=1)

In [ ]:
df_afstandn2 = get_dataframe("""
                                SELECT *
                                FROM proj_afval_netwerk.afv_rel_nodes_poi
                                """)

In [ ]:
df_afstandn2['split'] = df_afstandn2['bk_afv_rel_nodes_poi'].str.split('~')
df_afstandn2['x'] = df_afstandn2['split'].apply(lambda x: x[0]).astype('float').round().astype('int')
df_afstandn2['y'] = df_afstandn2['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
df_afstandn2['type'] = df_afstandn2['split'].apply(lambda x: x[2])
verblijfsobjecten = df_afstandn2[df_afstandn2['type'] != 'afval_cluster']
verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')


In [ ]:
temp = dfob.set_index(['bag', 'x', 'y']).join(verblijfsobjecten.set_index(['bag', 'x', 'y']), how='outer').reset_index()

In [ ]:
df_afstandn = get_distance_matrix()

In [ ]:
joined= temp.set_index('s1_afv_nodes').join(df_afstandn.set_index('naar_s1_afv_nodes'), how='outer')
joined = joined.reset_index()[['van_s1_afv_nodes', 'index', 'afstand', 'count']].\
         rename(columns={'index':'naar_s1_afv_nodes'}).sort_values(by='afstand').\
         reset_index().drop(['index'],axis=1).dropna()

In [ ]:
joined

In [ ]:
def distance_matrix_with_counts():
    """
    Function that tries to match table with addresses per poi with 
    information and subsequently with distance matrix to give back
    a distance matrix with the amount of households per addres poi.
    """
    
    dfob= get_dataframe("""
                    SELECT bk_votpand_cluster, COUNT(*)
                    FROM proj_afval_netwerk.rel_votpand_cluster_verblijfsobject
                    GROUP BY bk_votpand_cluster
                    """)
    
    dfob['split'] = dfob['bk_votpand_cluster'].str.split('~')
    dfob['bag'] = dfob['split'].apply(lambda x: x[0]).astype('int64')
    dfob['x'] = dfob['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
    dfob['y'] = dfob['split'].apply(lambda x: x[2]).astype('float').round().astype('int')
    dfob = dfob.drop(['split'], axis=1)
    
    df_afstandn2 = get_dataframe("""
                                SELECT *
                                FROM proj_afval_netwerk.afv_rel_nodes_poi
                                """)
    
    df_afstandn2['split'] = df_afstandn2['bk_afv_rel_nodes_poi'].str.split('~')
    df_afstandn2['x'] = df_afstandn2['split'].apply(lambda x: x[0]).astype('float').round().astype('int')
    df_afstandn2['y'] = df_afstandn2['split'].apply(lambda x: x[1]).astype('float').round().astype('int')
    df_afstandn2['type'] = df_afstandn2['split'].apply(lambda x: x[2])
    verblijfsobjecten = df_afstandn2[df_afstandn2['type'] != 'afval_cluster']
    verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')
    
    temp = dfob.set_index(['bag', 'x', 'y']).join(verblijfsobjecten.set_index(['bag', 'x', 'y']), how='outer').reset_index()
    df_afstandn = get_distance_matrix()
    joined= temp.set_index('s1_afv_nodes').join(df_afstandn.set_index('naar_s1_afv_nodes'), how='outer')
    joined = joined.reset_index()[['van_s1_afv_nodes', 'index', 'afstand', 'count']].\
             rename(columns={'index':'naar_s1_afv_nodes'}).sort_values(by='afstand').\
             reset_index().drop(['index'],axis=1).dropna()
    
    return joined

In [ ]:
joined = distance_matrix_with_counts()

### Try out

In [ ]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading(use_count=True)

In [ ]:
df_afstandn2

In [ ]:
joined_cluster_distance = joined.set_index('s1_afv_nodes').join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index().rename(columns={'index': 'van_s1_afv_nodes'})

In [ ]:
good_result_rich = add_shortest_distances_to_all_households(all_households, joined_cluster_distance, count=True)
good_result_rich = good_result_rich[good_result_rich['uses_container']]
good_result_rich

In [ ]:
aansluitingen = create_aansluitingen(good_result_rich, joined_cluster_distance, use_count=True)

In [ ]:

def calculate_penalties(good_result, aansluitingen, use_count=False):
    """
    This function calculates all the penalties associated with the candidate
    solution. It does this by calculating the number of times all constraints
    are violated and applies the weighing that is associated with all these
    violations

    Input:
    dataframe good_result containing per adress or adress poi the distance
    to the nearest container for all fractions.
    dataframe aansluitingen containing for all clusters the amount of containers
    per fraction, the amount of people using these containers and the percentage
    of occupancy compared to the norm

    Output:
    The sum of all different penalties as a single float
    """
    
    penalty1 = good_result[good_result['rest_afstand'] > 100]
    penalty2 = good_result[good_result['plastic_afstand'] > 150]
    penalty3 = good_result[good_result['papier_afstand'] > 150]
    penalty4 = good_result[good_result['glas_afstand'] > 150]
    penalty5 = good_result[good_result['textiel_afstand'] > 300]
    penalty6 = aansluitingen[aansluitingen['rest_perc'] > 100]
    penalty7 = aansluitingen[aansluitingen['plastic_perc'] > 100]
    penalty8 = aansluitingen[aansluitingen['papier_perc'] > 100]
    penalty9 = aansluitingen[aansluitingen['glas_perc'] > 100]
    penalty10 = aansluitingen[aansluitingen['textiel_perc'] > 100]
    
    if not use_count:
        penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1.shape[0])/good_result.shape[0] * 0.35
        penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2.shape[0])/good_result.shape[0] * 0.25
        penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3.shape[0])/good_result.shape[0] * 0.2
        penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4.shape[0])/good_result.shape[0] * 0.15
        penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5.shape[0])/good_result.shape[0] * 0.05
        penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result.shape[0] * 0.35 * 1000
        penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result.shape[0] * 0.25 * 1000
        penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result.shape[0] * 0.2 * 1000
        penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result.shape[0] * 0.15 * 1000
        penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result.shape[0] * 0.05 * 1000
        
    else:
        penalty1_sum = (penalty1['rest_afstand'].sum() - 100 * penalty1['count'].sum())/good_result['count'].sum() * 0.35
        penalty2_sum = (penalty2['plastic_afstand'].sum() - 150 * penalty2['count'].sum())/good_result['count'].sum() * 0.25
        penalty3_sum = (penalty3['papier_afstand'].sum() - 150 * penalty3['count'].sum())/good_result['count'].sum() * 0.2
        penalty4_sum = (penalty4['glas_afstand'].sum() - 150 * penalty4['count'].sum())/good_result['count'].sum() * 0.15
        penalty5_sum = (penalty5['textiel_afstand'].sum() - 300 * penalty5['count'].sum())/good_result['count'].sum() * 0.05
        penalty6_sum = (penalty6['poi_rest'] - (penalty6['rest'] * 100)).sum()/ good_result['count'].sum() * 0.35 * 1000
        penalty7_sum = (penalty7['poi_plastic'] - (penalty7['plastic'] * 200)).sum()/ good_result['count'].sum() * 0.25 * 1000
        penalty8_sum = (penalty8['poi_papier'] - (penalty8['papier'] * 200)).sum()/ good_result['count'].sum() * 0.2 * 1000
        penalty9_sum = (penalty9['poi_glas'] - (penalty9['glas'] * 200)).sum()/ good_result['count'].sum() * 0.15 * 1000
        penalty10_sum = (penalty10['poi_textiel'] - (penalty10['textiel'] * 750)).sum()/ good_result['count'].sum() * 0.05 * 1000
        
    total_penalties = sum([penalty1_sum, penalty2_sum, penalty3_sum, penalty4_sum, penalty5_sum,\
                           penalty6_sum, penalty7_sum, penalty8_sum, penalty9_sum, penalty10_sum])
    return total_penalties


In [ ]:
calculate_penalties(good_result_rich, aansluitingen, use_count=True)

### Initial test for correctly loading data using count per poi

In [2]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading(use_count=True)

API data loaded
a
DB relation POIs loaded
b
Table all households created
API and DB joined
containers per cluster determined


../Code\loading_data.py:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten['split'].apply(lambda x: x[3]).astype('int64')


distance matrix loaded


In [4]:
joined, joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = \
best_of_random(50, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)
hill_dict, best_solution = hillclimber(1000, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, parameter='penalties')

Average distance is : 177.29976726577087
Penalites are: 288.2499546710465
Average distance is : 177.84272863295124
Penalites are: 293.0649943473652
Average distance is : 178.48921167869267
Penalites are: 291.74643907709043
Average distance is : 178.1243698954531
Penalites are: 287.77676966019015
Average distance is : 180.26088843943268
Penalites are: 295.12363870975713
Average distance is : 181.79668032727832
Penalites are: 297.00572417899053
Average distance is : 178.64827314387628
Penalites are: 291.2219657555895
Average distance is : 180.26332079923517
Penalites are: 291.25061653498096
Average distance is : 178.9856994009263
Penalites are: 294.77532013300174
Average distance is : 180.32581783468663
Penalites are: 295.733855449905
Average distance is : 178.80208943309162
Penalites are: 295.5639363623373
Average distance is : 177.82671833938568
Penalites are: 297.57473047507284
Average distance is : 178.4021414985456
Penalites are: 293.4712934428121
Average distance is : 181.744541679

Average distance is : 178.17013951444733
Penalites are: 288.0008290912037
288.0008290912037 287.4380122786924
Average distance is : 178.11433674781014
Penalites are: 287.93775173173367
287.93775173173367 287.4380122786924
Average distance is : 178.14825340682899
Penalites are: 287.71533186802026
287.71533186802026 287.4380122786924
Average distance is : 178.13261410972723
Penalites are: 287.7513818287438
287.7513818287438 287.4380122786924
Average distance is : 178.13632189728617
Penalites are: 287.6995031119021
287.6995031119021 287.4380122786924
Average distance is : 178.09001829937444
Penalites are: 287.6062863169691
287.6062863169691 287.4380122786924
Average distance is : 178.18658163028698
Penalites are: 287.73166751811385
287.73166751811385 287.4380122786924
Average distance is : 178.13449603767603
Penalites are: 287.77826081887247
287.77826081887247 287.4380122786924
Average distance is : 178.136909919623
Penalites are: 287.85974816801524
287.85974816801524 287.4380122786924
Av

Average distance is : 178.09906183305753
Penalites are: 287.7721063100476
287.7721063100476 287.148620845075
Average distance is : 178.1453668939114
Penalites are: 287.54573658009366
287.54573658009366 287.148620845075
Average distance is : 178.15384371178314
Penalites are: 287.86006636206827
287.86006636206827 287.148620845075
Average distance is : 178.15773702618577
Penalites are: 287.9223319971105
287.9223319971105 287.148620845075
Average distance is : 178.1225671679291
Penalites are: 287.73886652741385
287.73886652741385 287.148620845075
Average distance is : 178.18029419755095
Penalites are: 288.02251433864524
288.02251433864524 287.148620845075
Average distance is : 178.04453679209433
Penalites are: 287.8210687529869
287.8210687529869 287.148620845075
Average distance is : 178.1308029189306
Penalites are: 287.7979302403828
287.7979302403828 287.148620845075
Average distance is : 178.1424208440752
Penalites are: 287.8280711754915
287.8280711754915 287.148620845075
Average distanc

Average distance is : 178.12268860518387
Penalites are: 287.7405498288865
287.7405498288865 287.148620845075
Average distance is : 178.21888362296434
Penalites are: 287.7196057291409
287.7196057291409 287.148620845075
Average distance is : 178.12163202346804
Penalites are: 287.63047243645735
287.63047243645735 287.148620845075
Average distance is : 178.14893621122354
Penalites are: 287.88877173799597
287.88877173799597 287.148620845075
Average distance is : 178.15275310221014
Penalites are: 287.7401953302242
287.7401953302242 287.148620845075
Average distance is : 178.13312686160754
Penalites are: 287.91135885065614
287.91135885065614 287.148620845075
Average distance is : 178.1310943362181
Penalites are: 288.32727080003076
288.32727080003076 287.148620845075
Average distance is : 178.1243551828585
Penalites are: 287.79811177990916
287.79811177990916 287.148620845075
Average distance is : 178.1573257638519
Penalites are: 288.09786570919266
288.09786570919266 287.148620845075
Average di

Average distance is : 177.93866798957072
Penalites are: 287.5752897502239
287.5752897502239 287.148620845075
Average distance is : 178.14142709322672
Penalites are: 287.67704715781434
287.67704715781434 287.148620845075
Average distance is : 178.1203303265517
Penalites are: 287.8780760658196
287.8780760658196 287.148620845075
Average distance is : 178.12372706968387
Penalites are: 287.87100216432384
287.87100216432384 287.148620845075
Average distance is : 178.05405236947905
Penalites are: 287.7000547096016
287.7000547096016 287.148620845075
Average distance is : 178.18272086529504
Penalites are: 287.77528571408953
287.77528571408953 287.148620845075
Average distance is : 178.13902800529826
Penalites are: 287.676913738787
287.676913738787 287.148620845075
Average distance is : 178.134153714694
Penalites are: 287.82493999631953
287.82493999631953 287.148620845075
Average distance is : 178.09706063558951
Penalites are: 287.6948963500381
287.6948963500381 287.148620845075
Average distance

...

Average distance is : 178.13668752763823
Penalites are: 287.78784764176595
287.78784764176595 287.148620845075
Average distance is : 178.12301369381052
Penalites are: 287.9869500928352
287.9869500928352 287.148620845075
Average distance is : 178.00827924562543
Penalites are: 287.50670730679013
287.50670730679013 287.148620845075
Average distance is : 178.13003324557283
Penalites are: 287.81390657976414
287.81390657976414 287.148620845075
Average distance is : 178.0744725771507
Penalites are: 287.77494731052553
287.77494731052553 287.148620845075
Average distance is : 178.1321716664813
Penalites are: 287.7005274289591
287.7005274289591 287.148620845075
Average distance is : 178.12551802207858
Penalites are: 287.8427652285449
287.8427652285449 287.148620845075
Average distance is : 178.1453961600556
Penalites are: 287.8624718871365
287.8624718871365 287.148620845075
Average distance is : 178.01154701661042
Penalites are: 287.71625186219745
287.71625186219745 287.148620845075
Average dist

Average distance is : 178.14505711215654
Penalites are: 288.21265627489106
288.21265627489106 287.148620845075
Average distance is : 178.14060958268698
Penalites are: 287.8789601646314
287.8789601646314 287.148620845075
Average distance is : 178.21410128926124
Penalites are: 288.0154752448597
288.0154752448597 287.148620845075
Average distance is : 178.10378473749694
Penalites are: 287.74696751866315
287.74696751866315 287.148620845075
Average distance is : 178.14522534707496
Penalites are: 288.2297799789501
288.2297799789501 287.148620845075
Average distance is : 178.12077972314722
Penalites are: 287.4957382912279
287.4957382912279 287.148620845075
Average distance is : 178.18979618150072
Penalites are: 288.0444665463654
288.0444665463654 287.148620845075
Average distance is : 178.13097427766735
Penalites are: 287.85837162373724
287.85837162373724 287.148620845075
Average distance is : 178.10905230553072
Penalites are: 287.7463524950149
287.7463524950149 287.148620845075
Average dista

Average distance is : 178.10811525696272
Penalites are: 287.9013159899035
287.9013159899035 287.148620845075
Average distance is : 178.14373969799524
Penalites are: 287.64486559219523
287.64486559219523 287.148620845075
Average distance is : 178.10589407477
Penalites are: 287.7374801832534
287.7374801832534 287.148620845075
Average distance is : 178.09873609792413
Penalites are: 287.70139627725314
287.70139627725314 287.148620845075
Average distance is : 178.1469011122706
Penalites are: 287.65866901363967
287.65866901363967 287.148620845075
Average distance is : 178.17054672563478
Penalites are: 287.85472822217827
287.85472822217827 287.148620845075
Average distance is : 178.09819396819105
Penalites are: 287.69016324027837
287.69016324027837 287.148620845075
Average distance is : 178.14523727530613
Penalites are: 287.7654717731525
287.7654717731525 287.148620845075
Average distance is : 178.22260046112007
Penalites are: 287.88151599072745
287.88151599072745 287.148620845075
Average dis

Average distance is : 178.08735921749445
Penalites are: 287.6437682882615
287.6437682882615 287.148620845075
Average distance is : 178.12266766812283
Penalites are: 287.71761029262314
287.71761029262314 287.148620845075
Average distance is : 178.12163741451502
Penalites are: 287.77727654729927
287.77727654729927 287.148620845075
Average distance is : 178.2706891162824
Penalites are: 287.91190203413476
287.91190203413476 287.148620845075
Average distance is : 178.145556326975
Penalites are: 287.78232496717806
287.78232496717806 287.148620845075
Average distance is : 178.2128646113241
Penalites are: 287.7602537614739
287.7602537614739 287.148620845075
Average distance is : 178.08355655112152
Penalites are: 287.75375418857084
287.75375418857084 287.148620845075
Average distance is : 178.41695810770523
Penalites are: 287.6599968276726
287.6599968276726 287.148620845075
Average distance is : 178.24952842667778
Penalites are: 287.98524359903973
287.98524359903973 287.148620845075
Average dis

../Code\helper_functions.py:461: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)


In [ ]:
joined, joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = \
best_of_random(10, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True)
hill_dict, best_solution = hillclimber(500, joined, all_households, rel_poi_df, df_afstandn2, clean=True, use_count=True, parameter='penalties')
hill_df = pd.DataFrame.from_dict(hill_dict, orient='index')
hill_df.to_csv('hillclimber.csv')

In [40]:
def random_start_hillclimber(joined, all_households, rel_poi_df, df_afstandn2,\
                        clean=True, use_count=False, parameter = 'penalties'):
    i = int(input("How many random iterations?"))
    j = int(input("How many iterations hillclimber?"))
    to_save = bool(input("Do you want the results saved(True/False)?"))
    
    joined, joined_cluster_distance, good_result_rich, aansluitingen, \
        avg_distance, penalties = best_of_random(i, joined,all_households, \
        rel_poi_df, df_afstandn2, clean=clean, use_count=clean)
    
    hill_df, best_solution = hillclimber(j, joined, all_households, \
        rel_poi_df, df_afstandn2, clean=clean, use_count=use_count,\
        parameter=parameter, save=to_save)
    hill_df['best'].plot()
    return hill_df, best_solution

In [ ]:
hill_df, best_solution = random_start_hillclimber(best_solution, all_households, rel_poi_df, df_afstandn2, use_count=True)

How many random iterations?10
How many iterations hillclimber1000
Do you want the results saved(True/False)?True
Average distance is : 175.92423959952254
Penalites are: 284.6418728538666
Average distance is : 180.83890826639936
Penalites are: 296.43736606473414
Average distance is : 179.6845176869466
Penalites are: 294.99554428542393
Average distance is : 180.36451248202178
Penalites are: 294.8623262598049
Average distance is : 181.1888079215068
Penalites are: 294.3105771201552
Average distance is : 179.40925536837074
Penalites are: 294.2166310647326
Average distance is : 180.3553362486917
Penalites are: 297.6834460625704
Average distance is : 180.07898330753213
Penalites are: 296.76947380220395
Average distance is : 180.8373638735383
Penalites are: 296.4376652278797
Average distance is : 181.23280922481385
Penalites are: 295.9901446109676
Average distance is : 179.7941692246861
Penalites are: 294.20260359756503
***************************************
175.92423959952254 284.64187285386

Average distance is : 175.42315716505294
Penalites are: 283.0903994149718
283.0903994149718 282.8764071356963
Average distance is : 175.5006646565374
Penalites are: 282.9889505335293
282.9889505335293 282.8764071356963
Average distance is : 175.4938279181258
Penalites are: 282.87645253168904
282.87645253168904 282.8764071356963
Average distance is : 175.48557496096512
Penalites are: 282.76568450361873
282.76568450361873 282.8764071356963
Average distance is : 175.5288098410069
Penalites are: 282.8773336816952
282.8773336816952 282.76568450361873
Average distance is : 175.51224384226762
Penalites are: 283.1658534820166
283.1658534820166 282.76568450361873
Average distance is : 175.50396133667564
Penalites are: 282.7320458519302
282.7320458519302 282.76568450361873
Average distance is : 175.52282525444085
Penalites are: 282.7875887375201
282.7875887375201 282.7320458519302
Average distance is : 175.51286613626306
Penalites are: 282.7702837647802
282.7702837647802 282.7320458519302
Averag

Average distance is : 175.42657304709485
Penalites are: 280.6892943101511
280.6892943101511 280.51259588367617
Average distance is : 175.38442722609426
Penalites are: 280.50929736161964
280.50929736161964 280.51259588367617
Average distance is : 175.47933916003444
Penalites are: 280.5706559682164
280.5706559682164 280.50929736161964
Average distance is : 175.37298243111366
Penalites are: 280.4344657293449
280.4344657293449 280.50929736161964
Average distance is : 175.33276445319248
Penalites are: 280.43749451640554
280.43749451640554 280.4344657293449
Average distance is : 175.3758208866036
Penalites are: 280.4224668472096
280.4224668472096 280.4344657293449
Average distance is : 175.35153147088536
Penalites are: 280.17496467190534
280.17496467190534 280.4224668472096
Average distance is : 175.28469330309136
Penalites are: 279.29751031621566
279.29751031621566 280.17496467190534
Average distance is : 175.2852880248558
Penalites are: 279.1881290653059
279.1881290653059 279.2975103162156

Average distance is : 175.1047384713541
Penalites are: 276.2797278150552
276.2797278150552 276.63175293954464
Average distance is : 175.13134339363353
Penalites are: 276.38919299233424
276.38919299233424 276.2797278150552
Average distance is : 175.13913990953648
Penalites are: 276.4092989271795
276.4092989271795 276.2797278150552
Average distance is : 175.11909810660444
Penalites are: 276.105300481753
276.105300481753 276.2797278150552
Average distance is : 175.11857870092695
Penalites are: 276.17710158561
276.17710158561 276.105300481753
Average distance is : 175.00420726407663
Penalites are: 276.05074039171956
276.05074039171956 276.105300481753
Average distance is : 175.00422365754758
Penalites are: 276.05046798430044
276.05046798430044 276.05074039171956
Average distance is : 175.11877953404536
Penalites are: 276.1265766533823
276.1265766533823 276.05046798430044
Average distance is : 174.9856889101473
Penalites are: 276.026718638677
276.026718638677 276.05046798430044
Average dist

Average distance is : 175.13145549796093
Penalites are: 275.4063790522595
275.4063790522595 274.283972436533
Average distance is : 175.1759735880323
Penalites are: 274.333382661287
274.333382661287 274.283972436533
Average distance is : 175.19083365847297
Penalites are: 274.8064975335565
274.8064975335565 274.283972436533
Average distance is : 175.16854794490652
Penalites are: 274.3081355170605
274.3081355170605 274.283972436533
Average distance is : 175.1428846121573
Penalites are: 274.20409758244597
274.20409758244597 274.283972436533
Average distance is : 175.14621267332075
Penalites are: 274.2950959125154
274.2950959125154 274.20409758244597
Average distance is : 175.12709597884083
Penalites are: 274.22088436555
274.22088436555 274.20409758244597
Average distance is : 175.15955888496748
Penalites are: 274.26684449321175
274.26684449321175 274.20409758244597
Average distance is : 175.1688631366405
Penalites are: 274.10992212850226
274.10992212850226 274.20409758244597
Average distan

...

Average distance is : 174.53553543883737
Penalites are: 265.61487222626585
265.61487222626585 265.5257237897743
Average distance is : 174.56974202712652
Penalites are: 265.69122144828816
265.69122144828816 265.5257237897743
Average distance is : 174.55751152556687
Penalites are: 265.0981164096296
265.0981164096296 265.5257237897743
Average distance is : 174.54596988640012
Penalites are: 265.02291295439255
265.02291295439255 265.0981164096296
Average distance is : 174.66039658181856
Penalites are: 265.00535921831437
265.00535921831437 265.02291295439255
Average distance is : 174.6206092269515
Penalites are: 265.3772988917423
265.3772988917423 265.00535921831437
Average distance is : 174.66265071933137
Penalites are: 265.00329553950314
265.00329553950314 265.00535921831437
Average distance is : 174.67855830508805
Penalites are: 265.1052763365697
265.1052763365697 265.00329553950314
Average distance is : 174.65327882633588
Penalites are: 264.88484755208265
264.88484755208265 265.003295539

Average distance is : 174.5795616868958
Penalites are: 262.7908810670212
262.7908810670212 262.6579093711891
Average distance is : 174.552876841455
Penalites are: 262.7672662283979
262.7672662283979 262.6579093711891
Average distance is : 174.61627580587788
Penalites are: 263.1818530888476
263.1818530888476 262.6579093711891
Average distance is : 174.648819890116
Penalites are: 262.7382354511596
262.7382354511596 262.6579093711891
Average distance is : 174.5528393306662
Penalites are: 262.45527445864144
262.45527445864144 262.6579093711891
Average distance is : 174.533181731291
Penalites are: 262.40448802886146
262.40448802886146 262.45527445864144
Average distance is : 174.57218083912645
Penalites are: 262.3968618876413
262.3968618876413 262.40448802886146
Average distance is : 174.59262662666336
Penalites are: 262.5854462088419
262.5854462088419 262.3968618876413
Average distance is : 174.5742617264214
Penalites are: 262.5112279762363
262.5112279762363 262.3968618876413
Average dista

Average distance is : 174.62777391368795
Penalites are: 261.3893339119544
261.3893339119544 261.35750946622204
Average distance is : 174.55179993222944
Penalites are: 261.2288389410136
261.2288389410136 261.35750946622204
Average distance is : 174.57378841829586
Penalites are: 261.2553977722174
261.2553977722174 261.2288389410136
Average distance is : 174.5650818529009
Penalites are: 261.38724748135047
261.38724748135047 261.2288389410136
Average distance is : 174.54411108874808
Penalites are: 261.05320559953196
261.05320559953196 261.2288389410136
Average distance is : 174.47306868478262
Penalites are: 260.8575611507234
260.8575611507234 261.05320559953196
Average distance is : 174.47845405539448
Penalites are: 260.98971743196626
260.98971743196626 260.8575611507234
Average distance is : 174.47922522698255
Penalites are: 260.82814752227506
260.82814752227506 260.8575611507234
Average distance is : 174.51137185384596
Penalites are: 260.8750499870909
260.8750499870909 260.82814752227506

Average distance is : 174.52842264888383
Penalites are: 258.746787137823
258.746787137823 258.73121620482146
Average distance is : 174.50441088386023
Penalites are: 258.7620717322699
258.7620717322699 258.73121620482146
Average distance is : 174.51139392151794
Penalites are: 258.7291473771091
258.7291473771091 258.73121620482146
Average distance is : 174.55803903375295
Penalites are: 258.79918706311474
258.79918706311474 258.7291473771091
Average distance is : 174.53339785186097
Penalites are: 258.8780500188144
258.8780500188144 258.7291473771091
Average distance is : 174.45048795966625
Penalites are: 258.659550227558
258.659550227558 258.7291473771091
Average distance is : 174.44578240972487
Penalites are: 258.61955587050204
258.61955587050204 258.659550227558
Average distance is : 174.44256189079994
Penalites are: 258.64855904560227
258.64855904560227 258.61955587050204
Average distance is : 174.42983145454895
Penalites are: 258.6774637129446
258.6774637129446 258.61955587050204
Aver

Average distance is : 174.0102824279194
Penalites are: 256.47855478970456
256.47855478970456 256.39517296468114
Average distance is : 173.98022105115086
Penalites are: 256.45178381577824
256.45178381577824 256.39517296468114
Average distance is : 174.035015545478
Penalites are: 256.527765141465
256.527765141465 256.39517296468114
Average distance is : 174.0250618064539
Penalites are: 256.5391979150358
256.5391979150358 256.39517296468114
Average distance is : 174.02665524648887
Penalites are: 256.5177638148444
256.5177638148444 256.39517296468114
Average distance is : 174.048392721003
Penalites are: 256.43061254027236
256.43061254027236 256.39517296468114
Average distance is : 174.1853456835457
Penalites are: 256.4247373414853
256.4247373414853 256.39517296468114
Average distance is : 174.0499197339351
Penalites are: 256.4801442075021
256.4801442075021 256.39517296468114
Average distance is : 174.01568004634882
Penalites are: 256.49521067311576
256.49521067311576 256.39517296468114
Ave

In [43]:
def hillclimber(num_iterations, joined, all_households, rel_poi_df, df_afstandn2, mod_max = 5, parameter='score', complicated=True, clean=True, use_count=False, save=True):
    """
    Function to perform repeated hillclimber. This can be added as a building block
    directly to the standard solution, but also after for example a random algorithm.
    The results are to be seen.
    """
    joined_cluster_distance, good_result_rich, aansluitingen, avg_distance, penalties = analyze_candidate_solution(joined, all_households, rel_poi_df, df_afstandn2, clean=clean, use_count=use_count)

    if parameter == 'score':
        best = avg_distance + penalties
    if parameter == 'penalties':
        best = penalties

    hillclimber_dict = {}
    hillclimber_dict[0] = [avg_distance, penalties, best]

    r = copy.deepcopy(joined)
    for i in range(1, num_iterations+1):
        last = copy.deepcopy(r)
        fractions = ['rest', 'plastic', 'papier', 'glas', 'textiel']
        no_modifications = random.randint(1, mod_max)
#         print(no_modifications)
        for j in range(no_modifications):
            valid = False
            while not valid:
                location_a = random.randint(0, r.shape[0]-1)
                fraction_a = random.choice(fractions)
                location_b = random.randint(0, r.shape[0]-1)
                fraction_b = random.choice(fractions)

                if r.at[location_a, fraction_b] > 0 and r.at[location_b, fraction_a] > 0\
                                                    and fraction_a != fraction_b:
                    r.at[location_a, fraction_a] = int(r.at[location_a, fraction_a]) + 1
                    r.at[location_a, fraction_b] = int(r.at[location_a, fraction_b]) - 1
                    r.at[location_b, fraction_a] = int(r.at[location_a, fraction_b]) - 1
                    r.at[location_b, fraction_b] = int(r.at[location_a, fraction_b]) + 1
                    valid = True

        joined_cluster_distance2, good_result_rich2, aansluitingen2, avg_distance2, penalties2 = analyze_candidate_solution(r, all_households, rel_poi_df, df_afstandn2, clean=clean, use_count=use_count)
        hillclimber_dict[i] = [avg_distance2, penalties2, best, no_modifications]
        if parameter == 'score':
            print(avg_distance2+penalties2, best)
            if avg_distance2+penalties2 < best:
                best = avg_distance2+penalties2
            else:
                r = copy.deepcopy(last) # Undo modification
        if parameter == 'penalties':
            print(penalties2, best)
            if penalties2 < best:
                best = penalties2
            else:
                r = copy.deepcopy(last) # Undo modification

    hill_df = pd.DataFrame.from_dict(hillclimber_dict, orient='index')
    hill_df = hill_df.rename(columns={0:'avg_distance', 1:'penalties', 2:'best', 3:'amount of modifications'})

    if save:
        today = str(pd.datetime.now().date()) + '-' + str(pd.datetime.now().hour)
        hill_df.to_csv('hillclimber' + today + '.csv')
        r.to_csv('hillclimber_best_config' + today + '.csv')

    return hill_df, r